In [1]:
##授權允許Colab存取個人雲端資料夾
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##物件預測和存取控制後輸出禁止物件序數清單
# 導入yolov5庫
!pip install yolov5
import torch
from PIL import Image
from yolov5 import detect

def filter_forbidden_objects(user_level, object_list, control_list):
    forbidden_object_indices = []

    for i, obj in enumerate(object_list):
        object_name = obj['name']
        object_level = control_list.get(object_name, 0)

        if user_level < object_level:
            forbidden_object_indices.append(i)

    return forbidden_object_indices

# 控制清單
control_list = {"car": 0, "person": 1, "gun": 2, "knife": 2, "blood": 3}

# 使用者等级，默認為0
user_level = 0

# 調用函數獲取預測结果
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
image_path = '/content/drive/MyDrive/test.jpeg'
image = Image.open(image_path)
results = model(image)

# 提取物件名稱和座標
pred_classes = results.pandas().xyxy[0]['name']
pred_boxes = results.pandas().xyxy[0][['xmin', 'ymin', 'xmax', 'ymax']]

# 建構物件清單
object_list = []
for i in range(len(pred_classes)):
    obj = {
        'name': pred_classes[i],
        'xmin': pred_boxes.iloc[i]['xmin'],
        'ymin': pred_boxes.iloc[i]['ymin'],
        'xmax': pred_boxes.iloc[i]['xmax'],
        'ymax': pred_boxes.iloc[i]['ymax']
    }
    object_list.append(obj)

# 調用過濾方法
forbidden_object_indices = filter_forbidden_objects(user_level, object_list, control_list)
print(forbidden_object_indices)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-1 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


[0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


In [ ]:
##物件預測和存取控制後輸出禁止物件座標清單
# 導入yolov5庫
!pip install yolov5
import torch
from PIL import Image
from yolov5 import detect

def get_forbidden_object_coordinates(forbidden_object_indices, object_list):
    forbidden_object_coordinates = []

    for index in forbidden_object_indices:
        obj = object_list[index]
        coordinates = {
            'xmin': obj['xmin'],
            'ymin': obj['ymin'],
            'xmax': obj['xmax'],
            'ymax': obj['ymax']
        }
        forbidden_object_coordinates.append(coordinates)

    return forbidden_object_coordinates

# 控制清單
control_list = {"car": 0, "person": 1, "gun": 2, "knife": 2, "blood": 3}

# 使用者等級，默認為0
user_level = 0

# 調用函數獲取預測結果
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
image_path = '/content/drive/MyDrive/test.jpeg'
image = Image.open(image_path)
results = model(image)

# 提取物件名稱和座標
pred_classes = results.pandas().xyxy[0]['name']
pred_boxes = results.pandas().xyxy[0][['xmin', 'ymin', 'xmax', 'ymax']]

# 建構物件清單
object_list = []
for i in range(len(pred_classes)):
    obj = {
        'name': pred_classes[i],
        'xmin': pred_boxes.iloc[i]['xmin'],
        'ymin': pred_boxes.iloc[i]['ymin'],
        'xmax': pred_boxes.iloc[i]['xmax'],
        'ymax': pred_boxes.iloc[i]['ymax']
    }
    object_list.append(obj)

# 使用過濾方法
forbidden_object_indices = filter_forbidden_objects(user_level, object_list, control_list)

# 獲取禁止物件座標清單
forbidden_object_coordinates = get_forbidden_object_coordinates(forbidden_object_indices, object_list)

# 印出禁止物件座標清單
for coordinates in forbidden_object_coordinates:
    print(coordinates)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-1 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


{'xmin': 1661.2293701171875, 'ymin': 393.6761474609375, 'xmax': 1897.0943603515625, 'ymax': 1035.8004150390625}
{'xmin': 779.344970703125, 'ymin': 456.69927978515625, 'xmax': 918.1175537109375, 'ymax': 624.041748046875}
{'xmin': 1202.920166015625, 'ymin': 459.7204284667969, 'xmax': 1477.3912353515625, 'ymax': 1051.65087890625}
{'xmin': 1035.64892578125, 'ymin': 477.2907409667969, 'xmax': 1137.23681640625, 'ymax': 583.0849609375}
{'xmin': 1199.150634765625, 'ymin': 466.6261291503906, 'xmax': 1294.168212890625, 'ymax': 578.0234985351562}
{'xmin': 1540.5482177734375, 'ymin': 462.8733825683594, 'xmax': 1629.641357421875, 'ymax': 586.1890869140625}
{'xmin': 1358.428466796875, 'ymin': 409.6060791015625, 'xmax': 1454.754150390625, 'ymax': 628.0188598632812}
{'xmin': 1447.44287109375, 'ymin': 467.85479736328125, 'xmax': 1543.17626953125, 'ymax': 587.184326171875}
{'xmin': 926.185546875, 'ymin': 460.7648010253906, 'xmax': 1020.3761596679688, 'ymax': 621.06591796875}
{'xmin': 1122.4512939453125,

In [2]:
##包含模型預測、簡易存取控制和特定物件屏蔽
# 導入yolov5庫
!pip install yolov5
import torch
from PIL import Image
from yolov5 import detect

#屏蔽功能依賴套件
import cv2
import numpy as np

# 完成預測並存取控制
def filter_forbidden_objects(user_level, object_list, control_list):
    forbidden_object_indices = []

    for i, obj in enumerate(object_list):
        object_name = obj['name']
        object_level = control_list.get(object_name, 0)

        if user_level < object_level:
            forbidden_object_indices.append(i)

    return forbidden_object_indices

def get_forbidden_object_coordinates(forbidden_object_indices, object_list):
    forbidden_object_coordinates = []

    for index in forbidden_object_indices:
        obj = object_list[index]
        coordinates = {
            'xmin': obj['xmin'],
            'ymin': obj['ymin'],
            'xmax': obj['xmax'],
            'ymax': obj['ymax']
        }
        forbidden_object_coordinates.append(coordinates)

    return forbidden_object_coordinates

# 控制清單
control_list = {"car": 0, "person": 1, "gun": 2, "knife": 2, "blood": 3}

# 使用者等級，默認為0
user_level = 0

# 調用函數獲取預測结果
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
image_path = '/content/drive/MyDrive/test.jpeg'
image = Image.open(image_path)
results = model(image)

# 提取物件名稱和座標
pred_classes = results.pandas().xyxy[0]['name']
pred_boxes = results.pandas().xyxy[0][['xmin', 'ymin', 'xmax', 'ymax']]

# 建構物件清單
object_list = []
for i in range(len(pred_classes)):
    obj = {
        'name': pred_classes[i],
        'xmin': pred_boxes.iloc[i]['xmin'],
        'ymin': pred_boxes.iloc[i]['ymin'],
        'xmax': pred_boxes.iloc[i]['xmax'],
        'ymax': pred_boxes.iloc[i]['ymax']
    }
    object_list.append(obj)

# 调用过滤方法
forbidden_object_indices = filter_forbidden_objects(user_level, object_list, control_list)

# 获取禁止物件坐标清单
forbidden_object_coordinates = get_forbidden_object_coordinates(forbidden_object_indices, object_list)

##針對禁止物件進行屏蔽
def put_mosaic(image, forbidden_object_coordinates):
    # 複製原始圖像
    image_copy = image.copy()

    # 遍歷邊界框列表
    for box in forbidden_object_coordinates:
        # 提取座標值
        x_min, y_min, x_max, y_max = box['xmin'], box['ymin'], box['xmax'], box['ymax']

        # 將座標值轉換為整數
        x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)

        # 提取要進行馬賽克的區域
        mosaic_region = image_copy.crop((x_min, y_min, x_max, y_max))

        # 縮小馬賽克區域
        scale_factor = 0.1
        small_mosaic = mosaic_region.resize((round((x_max - x_min) * scale_factor), round((y_max - y_min) * scale_factor)), Image.NEAREST)

        # 放大馬賽克區域
        enlarge_factor = 1.5
        enlarge_size = round((x_max - x_min) * (enlarge_factor - 1) / 2)
        enlarged_mosaic = small_mosaic.resize((x_max - x_min + 2 * enlarge_size, y_max - y_min + 2 * enlarge_size), Image.NEAREST)

        # 將馬賽克區域放回原始圖像
        image_copy.paste(enlarged_mosaic, (x_min - enlarge_size, y_min - enlarge_size))

    return image_copy

# 讀取圖像
image_path = '/content/drive/MyDrive/test.jpeg'
image = Image.open(image_path)

# 進行馬賽克處理
output_image = put_mosaic(image, forbidden_object_coordinates)

# 保存圖像
output_path = '/content/drive/MyDrive/test_mosaic.jpg'
output_image.save(output_path)
print('已保存馬賽克後的圖像：', output_path)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.3/953.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.6/612.6 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 18.2 MB/s et

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-7-2 Python-3.10.12 torch-2.0.1+cu118 CPU



[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/pyparsing-3.1.0.dist-info/METADATA'


100%|██████████| 14.1M/14.1M [00:00<00:00, 56.1MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


已保存馬賽克後的圖像： /content/drive/MyDrive/test_mosaic.jpg
